In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import ultraplot as uplt
from scipy import stats
from Py6S import *
import numpy.ma as ma

In [ ]:
## Define the linear mixing spectral model
## Nonlinear optimization is performed with respect to a simple L^2 loss
from scipy import constants as c
from scipy.interpolate import make_splrep
from mixture_model import SpectralMixtureModel, estimate_params_unconstrained

In [ ]:
SixS.test()

# Hotspot Identification

In [ ]:
pal_rad_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_RDN_3f4aef90_RDN.nc'
pal_mask_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_RDN_3f4aef90_BANDMASK.nc'
pal_ds = xr.open_datatree(pal_rad_path)
pal_ds

In [ ]:
pal_obs_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_ORT_8827a51f_OBS.nc'
obs_ds = xr.open_datatree(pal_obs_path) # Observational parameters
obs_ds 

In [ ]:
samples_coords = np.arange(1234)
lines_coords = np.arange(1280)
# Assign dummy coordinates
pal_radiance = pal_ds.radiance.radiance.assign_coords({'samples':samples_coords, 'lines':lines_coords})
pal_radiance

In [ ]:
# observation parameters
obs_params = obs_ds.observation_parameters.to_dataset().assign_coords({'samples':samples_coords, 'lines':lines_coords})
obs_params

# Sample Images/Plots

In [ ]:
# Generate an RGB image
def normalize(band):
    band_min = band.min()
    band_max = band.max()
    return (band - band_min) / (band_max - band_min)

red_ = pal_radiance.sel(wavelength=700, method='nearest')
green_ = pal_radiance.sel(wavelength=550, method='nearest')
blue_ = pal_radiance.sel(wavelength=400, method='nearest')

red = normalize(red_)
green = normalize(green_) 
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
red_ = pal_radiance.sel(wavelength=2200, method='nearest')
green_ = pal_radiance.sel(wavelength=700, method='nearest')
blue_ = pal_radiance.sel(wavelength=550, method='nearest')

red = normalize(red_)
green = normalize(green_)
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)

In [ ]:
# Visualize the observation parameters
fig, ax = uplt.subplots(ncols=2)
obs_params.path_length.plot(ax=ax[0])
ax[0].format(
    title='Path Length',
    yreverse=True,
)

obs_params.cosine_i.plot(ax=ax[1])
ax[1].format(
    title='cosine_i',
    yreverse=True,
)
plt.savefig('plots/view_params.png')

In [ ]:
# Visualize the observation parameters
fig, axs = uplt.subplots(nrows=2, ncols=2)

ax = axs[0,0]
obs_params.to_sun_zenith.plot(ax=ax)
ax.format(
    title='Solar Zenith Angle',
    yreverse=True,
)

ax = axs[0,1]
obs_params.to_sun_azimuth.plot(ax=ax)
ax.format(
    title='Solar Azimuth Angle',
    yreverse=True,
)

ax = axs[1,0]
obs_params.to_sensor_zenith.plot(ax=ax)
ax.format(
    title='Sensor Zenith Angle',
    yreverse=True,
)

ax = axs[1,1]
obs_params.to_sensor_azimuth.plot(ax=ax)
ax.format(
    title='Sensor Azimuth Angle',
    yreverse=True,
)
plt.savefig('plots/view_angles.png')

Both the solar zenith angle and azimuth angle can be treated as approximately constant throughout the read area.
As for the sensor zenith angle and azimuth angle, there is considerable variation in the angles across the sample dimension, because the sensor is read from a pushbroom sensor on an aircraft. Therefore from an aircraft altitude, the line-of-sight angle from left-to-right across the pushbroom varies significantly.

# Simple Atmospheric Correction

In [ ]:
import inspect

In [ ]:
# From path length we assume an altitude of 5km
# Create a 6S object from the viewing 
view = SixS()
view.altitudes.set_target_sea_level()
view.altitudes.set_sensor_custom_altitude(
    altitude=5 # 5km altitude
)
# Set atmospheric profiles; Data from Table 2-2 in http://www.exelisvis.com/docs/FLAASH.html
# For Los Angeles at a 34N latitude, recommended to set MidlatitudeSummer
view.atmos_profile = AtmosProfile.PredefinedType(AtmosProfile.MidlatitudeSummer)
# Aerosol profiles with pre-defined type
view.aero_profile = AeroProfile.PredefinedType(AeroProfile.BiomassBurning)
# Configure the sensor geometry
view.geometry = Geometry.User()
view.geometry.solar_z = 57.75
view.geometry.solar_a = 197.55
view.geometry.view_z = 0 # Assume fully Nadir-viewing
view.geometry.view_a = 0 # Consistent with Nadir-view

In [ ]:
# Set the wavelengths for the simulation
wavelengths = pal_radiance.wavelength.values/1000 # Wavelengths in micrometres
wv, res = SixSHelpers.Wavelengths.run_wavelengths(view, wavelengths)

In [ ]:
inspect.getmembers(res[0])

In [ ]:
# This produces an array of Py6S output objects arranged by wavelength
res_T_gas = np.array([s.total_gaseous_transmittance for s in res]) # Total gaseous transmittance
res_T_water = np.array([s.transmittance_water.total for s in res]) # Water vapour transmittance
res_T_up = np.array([s.transmittance_total_scattering.upward for s in res]) # Upward scattering transmittance
res_T_down = np.array([s.transmittance_total_scattering.downward for s in res]) # Downward scattering transmittance
# Get atmospheric intrinsic reflectance, which is scattering path radiance multiplied by transmittance
res_atm_ref = np.array([s.atmospheric_intrinsic_reflectance for s in res])


In [ ]:
np.argwhere(np.isnan(res_T_up))

In [ ]:
# Linearly interp the transmittance gaps
def interp_nans(array):
    xp = np.arange(len(array))
    # Get nans
    notnan = ~np.isnan(array)
    return np.interp(xp, xp[notnan], array[notnan])

res_T_up = interp_nans(res_T_up)
res_T_down = interp_nans(res_T_down)
res_atm_ref = interp_nans(res_atm_ref)

In [ ]:
fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_up, label='upward transmittance')
ax.plot(wv, res_T_down, label='downward transmittance')
ax.plot(wv, res_T_gas, label='gaseous transmittance')
ax.plot(wv, res_atm_ref, label='atmospheric intrinsic reflectance')
fig.legend(loc='b', ncols=2)


In [ ]:
# Choose a random pixel to do atmospheric correction as a test
pixel = pal_radiance.sel(lines=100, samples=1000)


fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_up, label='upward transmittance')
ax.plot(wv, res_T_down, label='downward transmittance')
ax.plot(wv, res_T_gas, label='gaseous transmittance')
ax.plot(wv, res_atm_ref, label='atmospheric intrinsic reflectance')
ax.plot(wv, pixel, label='pixel spectrum', c='k')
ax.format(suptitle='py6S output')
fig.legend(loc='b', ncols=2)
plt.savefig('plots/6S_results.png')

In [ ]:
# From the notes; relation of TOA radiance with ground radiance;
# L_TOA = (L_ground/(1-S rho) T_up T_down + L_path) Tg
# Neglecting the contribution from spherical albedo
# L_TOA = L_ground T_up T_down Tg + atm_intr_refl
# L_ground = (L_TOA - atm_intr_refl)/(T_up T_down T_g)

pixel_ground = (pixel - res_atm_ref) / (res_T_up * res_T_down * res_T_gas)

In [ ]:
fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_gas, label='Gas trasmittance')
ax.plot(wv, pixel, label='pixel spectrum', c='k')
ax.plot(wv, pixel_ground, label='corrected pixel spectrum', c='r')
ax.axhline(0.6, c='k', linestyle='--', lw=.5)
fig.legend(loc='b', ncols=2)
ax.format(
    ylim=(0,5),
    suptitle='Basic atmospheric correction for test pixel'
)

In [ ]:
# We define problematic bands and mask them
# Mask all regions with less than 0.6 gaseous transmittance
# This is a tradeoff to prevent large losses due to absorption spikes
T_gas_cutoff = 0.6
wv_mask = res_T_gas < T_gas_cutoff

In [ ]:
fig, ax = uplt.subplots(refwidth=6, refaspect=(2,1))
ax.plot(wv, res_T_gas, label='Gas trasmittance')
ax.plot(wv, pixel, label='pixel spectrum', c='k')
ax.plot(wv, pixel_ground.where(~wv_mask), label='corrected pixel spectrum', c='r')
ax.axhline(0.6, c='k', linestyle='--', lw=.5)
fig.legend(loc='b', ncols=2)
ax.format(
    ylim=(0,5),
    suptitle='Basic atmospheric correction for test pixel'
)
plt.savefig('plots/single_pixel_correction.png')

In [ ]:
pixel_ground.where(~wv_mask).sel(wavelength=2430, method='nearest')

## Dataset correction

In [ ]:
pal_radiance.coords['wavelength']

In [ ]:
# For xarray broadcasting, all of the terms need to be cast into dataarrays
res_atm_ref_da = xr.DataArray(
    res_atm_ref,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)
res_T_up_da = xr.DataArray(
    res_T_up,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)
res_T_down_da = xr.DataArray(
    res_T_down,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)
res_T_gas_da = xr.DataArray(
    res_T_gas,
    dims=('wavelength'),
    coords={'wavelength': pal_radiance.coords['wavelength']}
)

In [ ]:
wv_mask = res_T_gas_da < T_gas_cutoff

In [ ]:
# Save everything as a dataset
sixs_ds = xr.merge([wv_mask.rename('mask'), 
    res_atm_ref_da.rename('atmospheric_path_reflectance'), 
    res_T_up_da.rename('upward_atmospheric_transmittance'),
    res_T_down_da.rename('downward_atmospheric_transmittance'),
    res_T_gas_da.rename('gaseous_transmittance'),
    ])

#sixs_ds.to_netcdf('datasets/sixs_output.nc')

In [ ]:
ground_radiance = (pal_radiance - res_atm_ref_da) / (res_T_up_da * res_T_down_da)

In [ ]:
red_ = ground_radiance.sel(wavelength=700, method='nearest')
green_ = ground_radiance.sel(wavelength=550, method='nearest')
blue_ = ground_radiance.sel(wavelength=400, method='nearest')

red = normalize(red_)
green = normalize(green_) 
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)
plt.savefig('plots/RGB_image.png')

In [ ]:
red_ = ground_radiance.sel(wavelength=2200, method='nearest')
green_ = ground_radiance.sel(wavelength=700, method='nearest')
blue_ = ground_radiance.sel(wavelength=550, method='nearest')

red = normalize(red_)
green = normalize(green_)
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(5, 5))
plt.imshow(rgb_image)
plt.savefig('plots/false_colour_image.png')

This simplistic atmospheric correction is very limited because it does not take into account:

- Topography (cos_i) angle
- Gaseous transmittance
- Aerosol scattering from burning plumes
- View angle and inhomogenous path lengths

# HFDI Hotspot

In [ ]:
# Calculate the HFDI index
# Remember that HFDI was designed to be robust against atmospheric absorption
pal_rad_2430 = pal_radiance.sel(wavelength=slice(2420,2440)).mean(dim='wavelength')
pal_rad_2060 = pal_radiance.sel(wavelength=slice(2050,2070)).mean(dim='wavelength')
pal_HFDI = (pal_rad_2430 - pal_rad_2060)/(pal_rad_2430 + pal_rad_2060)

In [ ]:
fig, ax = uplt.subplots(refwidth=6)
pal_HFDI.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI Index'
)
plt.savefig('plots/HFDI_index.png')

In [ ]:
bins = np.linspace(-0.4, 0.4, 500)
fig, ax = uplt.subplots(refwidth=6, refaspect=(3,1))
_ = pal_HFDI.plot.hist(bins=bins, ax=ax)
ax.format(
    suptitle='Distribution of pixel HFDI'
)
plt.savefig('plots/HFDI_distribution.png')

This distribution looks like a skew-normal with a long-tail anomaly. Assume that background pixels follow a skew-normal distribution, and use this to determine an appropriate threshold.

In [ ]:
# Define a fire mask
fire_mask = (pal_HFDI>0.1)

In [ ]:
fig, ax = uplt.subplots(refwidth=3)
fire_mask.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.plot(500, 1050, marker='o', c='r', s=1)
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI > 0.01'
)

In [ ]:
# Now find the region in shade
fig, ax = uplt.subplots()
(np.absolute(obs_params.cosine_i) < 0.05).plot(ax=ax)
ax.plot(800, 640, marker='o', c='r', s=1)

# Test with individual pixels

In [ ]:
# choose a fire pixel
fire_pixel = ground_radiance.sel(samples=500, lines=1050)
print('Fire pixel HFDI:', pal_HFDI.sel(samples=500, lines=1050).values)
print('Unburnt pixel cosi:', obs_params.cosine_i.sel(samples=500, lines=1050).values)
# choose an unburnt pixel
unburnt_pixel = ground_radiance.sel(samples=200, lines=600)
print('Unburnt pixel HFDI:', pal_HFDI.sel(samples=200, lines=600).values)
print('Unburnt pixel cosi:', obs_params.cosine_i.sel(samples=200, lines=600).values)
# choose a pixel in shade
shade_pixel = ground_radiance.sel(samples=800, lines=640)
print('Shade pixel HFDI:', pal_HFDI.sel(samples=800, lines=640).values)
print('Shade pixel cosi:', obs_params.cosine_i.sel(samples=800, lines=640).values)

In [ ]:
# Plot the test spectra
fig, ax = uplt.subplots(refwidth=4, refaspect=(2,1))
ax.plot(fire_pixel.where(~wv_mask), label='Fire pixel')
ax.plot(unburnt_pixel.where(~wv_mask), label='Unburnt pixel')
ax.plot(shade_pixel.where(~wv_mask), label='Shaded pixel')
ax.legend()

In [ ]:
## Define the linear mixing spectral model
## Nonlinear optimization is performed with respect to a simple L^2 loss
from scipy import constants as c
from scipy.interpolate import make_splrep
from mixture_model import SpectralMixtureModel, estimate_params_unconstrained

In [ ]:
bkg_spectra_1 = unburnt_pixel.where(~wv_mask).dropna(dim='wavelength')
bkg_spectra_1

In [ ]:
fire_spectra = fire_pixel.where(~wv_mask).dropna(dim='wavelength')

In [ ]:
# Create a spectral mixture model instance
# Retrieve the unpurnt pixel spectra as the bkg spectra
mask_lambds = bkg_spectra_1.wavelength.values
simple_model = SpectralMixtureModel(
    n_fire=1,
    n_bkg=1,
    bkg_spectra_lis=[(mask_lambds, bkg_spectra_1.values)],
    SI=False,    
)

In [ ]:
# Invert the parameters
result, params = estimate_params_unconstrained(
    model=simple_model,
    lambd=mask_lambds,
    target=fire_spectra,
    x0=np.array([1., 0.0,]), # Temperature in terms of 1000Kd
)

In [ ]:
fitted_spectra = simple_model.total_radiance(
    mask_lambds, params[0], params[1], params[2]
)

In [ ]:
# Plot the test spectra
fig, ax = uplt.subplots(refwidth=4, refaspect=(2,1))
ax.plot(fire_pixel.where(~wv_mask), label=f'Fire pixel')
ax.plot(mask_lambds, fitted_spectra, label=f'Fitted spectra, frac={params[1][0]:.3}, T={params[0][0]:.4}K')
ax.legend(ncols=1)